In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [20]:
data = pd.read_excel('Data_v.4.xlsx')
data=data.copy()

In [21]:
# Extract features
features = ['Category', 'City', 'Rating', 'Price']
X = data[features].copy()

# Encode categorical features
le_category = LabelEncoder()
le_city = LabelEncoder()
X['Category'] = le_category.fit_transform(X['Category'])
X['City'] = le_city.fit_transform(X['City'])

# Scale numerical features
scaler = MinMaxScaler()
X[['Rating', 'Price']] = scaler.fit_transform(X[['Rating', 'Price']])

# Convert to numpy array
X = X.values

# Step 2: Feature Encoding
# One-hot encode Category and City
num_categories = len(le_category.classes_)
num_cities = len(le_city.classes_)
X_encoded = np.zeros((X.shape[0], num_categories + num_cities + 2))
X_encoded[:, :num_categories] = tf.keras.utils.to_categorical(X[:, 0], num_classes=num_categories)
X_encoded[:, num_categories:num_categories+num_cities] = tf.keras.utils.to_categorical(X[:, 1], num_classes=num_cities)
X_encoded[:, -2:] = X[:, 2:]  # Rating and Price


In [22]:
# Step 3: Model Architecture
input_dim = X_encoded.shape[1]
hidden_dim = 64
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(hidden_dim, activation='relu'),
    tf.keras.layers.Dense(input_dim, activation='sigmoid')
])
model.compile(optimizer='adam', loss='mse')

# Step 4: Training
X_train, X_val = train_test_split(X_encoded, test_size=0.2, random_state=42)
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val), verbose=1)


Epoch 1/50
16/16 [==============================] - 1s 29ms/step - loss: 0.2232 - val_loss: 0.1989
Epoch 2/50
16/16 [==============================] - 0s 11ms/step - loss: 0.1716 - val_loss: 0.1351
Epoch 3/50
16/16 [==============================] - 0s 13ms/step - loss: 0.1039 - val_loss: 0.0743
Epoch 4/50
16/16 [==============================] - 0s 17ms/step - loss: 0.0623 - val_loss: 0.0546
Epoch 5/50
16/16 [==============================] - 0s 14ms/step - loss: 0.0511 - val_loss: 0.0476
Epoch 6/50
16/16 [==============================] - 0s 10ms/step - loss: 0.0453 - val_loss: 0.0416
Epoch 7/50
16/16 [==============================] - 0s 12ms/step - loss: 0.0397 - val_loss: 0.0362
Epoch 8/50
16/16 [==============================] - 0s 23ms/step - loss: 0.0342 - val_loss: 0.0306
Epoch 9/50
16/16 [==============================] - 0s 16ms/step - loss: 0.0293 - val_loss: 0.0260
Epoch 10/50
16/16 [==============================] - 0s 15ms/step - loss: 0.0254 - val_loss: 0.0226
Epoch 11/

In [23]:
# Step 5: Create a Custom Model Class
class RecommendationModel(tf.keras.Model):
    def __init__(self, model, data, le_category, le_city, scaler, num_categories, num_cities):
        super(RecommendationModel, self).__init__()
        self.model = model
        self.data = data
        self.le_category = le_category
        self.le_city = le_city
        self.scaler = scaler
        self.num_categories = num_categories
        self.num_cities = num_cities
        self.input_dim = num_categories + num_cities + 2
        self.all_embeddings = None
    
    def compute_all_embeddings(self):
        X = self.data[['Category', 'City', 'Rating', 'Price']].copy()
        X['Category'] = self.le_category.transform(X['Category'])
        X['City'] = self.le_city.transform(X['City'])
        X[['Rating', 'Price']] = self.scaler.transform(X[['Rating', 'Price']])
        X_encoded = np.zeros((X.shape[0], self.input_dim))
        X_encoded[:, :self.num_categories] = tf.keras.utils.to_categorical(X['Category'], num_classes=self.num_categories)
        X_encoded[:, self.num_categories:self.num_categories+self.num_cities] = tf.keras.utils.to_categorical(X['City'], num_classes=self.num_cities)
        X_encoded[:, -2:] = X[['Rating', 'Price']]
        self.all_embeddings = self.model.predict(X_encoded)
    
    def get_feature_vector(self, attraction_name):
        attraction = self.data[self.data['Place_Name'] == attraction_name].iloc[0]
        features = [
            self.le_category.transform([attraction['Category']])[0],
            self.le_city.transform([attraction['City']])[0],
            attraction['Rating'],
            attraction['Price']
        ]
        encoded = np.zeros((1, self.input_dim))
        encoded[0, :self.num_categories] = tf.keras.utils.to_categorical(features[0], num_classes=self.num_categories)
        encoded[0, self.num_categories:self.num_categories+self.num_cities] = tf.keras.utils.to_categorical(features[1], num_classes=self.num_cities)
        encoded[0, -2:] = self.scaler.transform([features[2:]])
        return encoded
    
    def get_recommendations(self, attraction_name, top_k=10):
        if self.all_embeddings is None:
            self.compute_all_embeddings()
        
        query_vector = self.get_feature_vector(attraction_name)
        query_embedding = self.model.predict(query_vector)
        
        similarities = np.dot(self.all_embeddings, query_embedding.T).flatten()
        top_indices = similarities.argsort()[::-1][1:top_k+1]  # Exclude the query itself
        top_attractions = self.data.iloc[top_indices]['Place_Name'].tolist()
        top_similarities = similarities[top_indices].tolist()
        
        return list(zip(top_attractions, top_similarities))


In [24]:
# Step 6: Create an Instance of the Custom Model
recommendation_model = RecommendationModel(
    model=model,
    data=data,
    le_category=le_category,
    le_city=le_city,
    scaler=scaler,
    num_categories=num_categories,
    num_cities=num_cities
)

# Step 7: Save the Model
# Save the TensorFlow model
model.save('recommendation_model_weights.h5')

In [27]:
# Save other components
with open('recommendation_model_components.pkl', 'wb') as f:
    pickle.dump({
        'data': data,
        'le_category': le_category,
        'le_city': le_city,
        'scaler': scaler,
        'num_categories': num_categories,
        'num_cities': num_cities
    }, f)

In [28]:
# Step 8: Load the Model (this would typically be in a different script)
# Load the TensorFlow model
loaded_model = tf.keras.models.load_model('recommendation_model_weights.h5')

# Load other components
with open('recommendation_model_components.pkl', 'rb') as f:
    components = pickle.load(f)

# Recreate the RecommendationModel
loaded_recommendation_model = RecommendationModel(
    model=loaded_model,
    data=components['data'],
    le_category=components['le_category'],
    le_city=components['le_city'],
    scaler=components['scaler'],
    num_categories=components['num_categories'],
    num_cities=components['num_cities']
)


In [29]:
# Step 9: Use the Loaded Model for Recommendations
query_attraction = "Monumen Nasional"  # Replace with an actual attraction from your dataset
recommendations = loaded_recommendation_model.get_recommendations(query_attraction, top_k=10)
print(f"Top 10 recommendations similar to {query_attraction}:")
for attraction, similarity in recommendations:
    print(f"{attraction}: {similarity}")

1/1 [==============================] - 0s 64ms/step
Top 10 recommendations similar to Monumen Nasional:
Museum Bank Indonesia: 2.459745407104492
Museum Sasmita Loka Ahmad Yani: 2.459745407104492
Museum Kebangkitan Nasional: 2.459745407104492
Perpustakaan Nasional: 2.459695339202881
Monumen Nasional: 2.4406087398529053
Taman Ismail Marzuki: 2.440218448638916
Museum Nasional: 2.440218448638916
Museum Tengah Kebun: 2.4400830268859863
Istana Negara Republik Indonesia: 2.4400830268859863
Kota Tua: 2.4400830268859863


C:\Users\User\anaconda3\envs\capstone\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [30]:
recommendations

[('Museum Bank Indonesia', 2.459745407104492),
 ('Museum Sasmita Loka Ahmad Yani', 2.459745407104492),
 ('Museum Kebangkitan Nasional', 2.459745407104492),
 ('Perpustakaan Nasional', 2.459695339202881),
 ('Monumen Nasional', 2.4406087398529053),
 ('Taman Ismail Marzuki', 2.440218448638916),
 ('Museum Nasional', 2.440218448638916),
 ('Museum Tengah Kebun', 2.4400830268859863),
 ('Istana Negara Republik Indonesia', 2.4400830268859863),
 ('Kota Tua', 2.4400830268859863)]